In [ ]:
'''
data: MNIST
  input: 28x28 pixels of value (0->1)
  output: a model able to create mnist images

framework: tf.keras
model: vanilla GAN
  layers:
    discriminator: 512 relu (0.3 dropout) | 256 relu | 1 sigmoid
    generator: 128 relu | 256 relu | 512 relu | 28*28 sigmoid (pixel value 0-1)
  params: optimizer adam default learning rate 
  hyperparams: batch_size, noise_size
  algorithm: GAN-2 MLP

result: its did learn but not too good: hard to control, easy to overfit
  test: train gen until ~ 80 confident => train dis => repeat. Gen image can be recognized, just not goodlooking
'''

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import struct

with open("10kimages.idx3-ubyte", "rb") as file:
    magic, num, rows, cols = struct.unpack(">IIII", file.read(16))
    data = np.frombuffer(file.read(), dtype=np.uint8)
    features = data.reshape(num, rows * cols).astype(np.float32) / 255.0  # (10000, 784)

with open("10klabels.idx1-ubyte", "rb") as file:
    magic, num = struct.unpack(">II", file.read(8))
    labels = np.frombuffer(file.read(), dtype=np.uint8)  # (10000,)

indices = np.where(labels == 6)[0]
image = features[indices]

print(image.shape) # (958, 784)

In [ ]:
noise_size = 128
image_size = 28*28
batch_size = 958

In [ ]:
def models():
    global dismodel, disopti, genmodel, genopti

    dismodel = tf.keras.models.Sequential([
        tf.keras.Input(shape=(image_size,)),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid")])
    disopti = tf.keras.optimizers.Adam()

    genmodel = tf.keras.models.Sequential([
        tf.keras.Input(shape=(noise_size,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dense(image_size, activation="sigmoid")])
    genopti = tf.keras.optimizers.Adam()

models()

In [ ]:
def discriminator(imgs):
    return dismodel(imgs, training=True)

def generator():
    noise = np.random.rand(batch_size, noise_size)
    return genmodel(noise, training=True)

In [ ]:
def distrain(iterations):
    dislosses = []
    for iteration in range(iterations + 1):
        with tf.GradientTape() as tape:
            real_img = image[np.random.randint(0, image.shape[0], batch_size)]
            fake_img = generator()

            real_predict = discriminator(real_img)
            fake_predict = discriminator(fake_img)
            
            real_loss = -tf.reduce_mean(tf.math.log(real_predict + 1e-8))
            fake_loss = -tf.reduce_mean(tf.math.log((tf.ones_like(fake_predict) - fake_predict) + 1e-8))
            BinaryCrossEntropy = real_loss + fake_loss
            dislosses.append(BinaryCrossEntropy)
            print(f"iteration {iteration}: loss = {BinaryCrossEntropy}")

        disgrad = tape.gradient(BinaryCrossEntropy, dismodel.trainable_variables)
        disopti.apply_gradients(zip(disgrad, dismodel.trainable_variables))
    return dislosses

dislosses = distrain(0)

In [ ]:
def gentrain(iterations):
    genlosses = []
    for iteration in range(iterations + 1):
        with tf.GradientTape() as tape:
            fake_img = generator()
            fake_predict = discriminator(fake_img)

            BinaryCrossEntropy = tf.reduce_mean(tf.math.log((tf.ones_like(fake_predict) - fake_predict) + 1e-8))
            genlosses.append(BinaryCrossEntropy)
            if (iteration % 20 == 0):
                print(f"iteration {iteration}: loss = {BinaryCrossEntropy}")

        gengrad = tape.gradient(BinaryCrossEntropy, genmodel.trainable_variables)
        genopti.apply_gradients(zip(gengrad, genmodel.trainable_variables))
    return genlosses

genlosses = gentrain(200)

In [ ]:
def distest(rows, cols):
    data = image[np.random.randint(0, image.shape[0], batch_size)]
    real_predict = discriminator(data) 
    data = tf.reshape(data, (-1, 28, 28)) 
    
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 2, rows * 2))  
    
    for i in range(rows):  
        for j in range(cols):
            idx = i * cols + j
            img = data[idx]
            label = real_predict[idx]
            axes[i, j].imshow(img, cmap='gray')
            axes[i, j].set_title(f"Confident: {label}")
            axes[i, j].axis('off')
    plt.tight_layout()

distest(4, 8)

In [ ]:
def gentest(rows, cols):
    data = generator()
    real_predict = discriminator(data) 
    data = tf.reshape(data, (-1, 28, 28)) 
    
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 2, rows * 2))  
    
    for i in range(rows):
        for j in range(cols):
            idx = i * cols + j
            img = data[idx]
            label = real_predict[idx]
            axes[i, j].imshow(img, cmap='gray')
            axes[i, j].set_title(f"Confident: {label}")
            axes[i, j].axis('off')
    plt.tight_layout()

gentest(4, 8)

In [ ]:
def gen(rows, cols):
    data = generator()
    data = tf.reshape(data, (-1, 28, 28))

    fig, axes = plt.subplots(rows, cols, figsize=(cols * 2, rows * 2))

    for i, ax in enumerate(axes.flat):
        ax.imshow(data[i], cmap="gray")
        ax.axis("off")

gen(4, 8)

In [ ]:
genmodel.save("genmodel.keras")